In [1]:
from gototile.gaussian import gaussian_skymap
import healpy
from matplotlib import pyplot as plt
from matplotlib.figure import Figure
import cartopy.crs as ccrs
import cartopy
import gototile.skymaptools as smt
from matplotlib.colors import colorConverter
import numpy as np
import pandas as pd

In [13]:
def plot_skymap(trigno, ra, dec, radius, nside=64):
        %matplotlib qt
        """
        Plot the coverage tiling region for Fermi trigger 
        ----------
        trigno : integer
            Fermi GRB trigger number
        ra : float
            ra in decimal degrees
        dec : float
            declination in decimal degrees
        radius : float
            68% containment radius in decimal degrees
        nside : int, optional
            healpix nside parameter (must be a power of 2)
            default is 64

        Returns
        -------
        `~gototile.skymap.SkyMap``
            SkyMap object.
        """
        hdulist = gaussian_skymap(ra, dec, radius, nside)
        info = healpy.read_map(hdulist, h=True, field=None,
                               verbose=False, nest=None)
        skymap = info[0]
        
        """
        Plotting the gaussian skymap for given RA, Dec, Error
        """
        
        figure = plt.figure(figsize=(15, 15))
        axes = figure.add_subplot(1, 1, 1, projection=ccrs.Mollweide())
        geodetic = ccrs.Geodetic()
        npix = healpy.nside2npix(nside)
        ipix = np.arange(npix)
        thetas, phis = healpy.pix2ang(nside, ipix)
        ras = np.rad2deg(phis)%360
        decs = np.rad2deg(np.pi/2 - thetas%np.pi)
        axes.scatter(ras, decs, s=20, c=skymap,cmap='BuGn', alpha=1, 
                     linewidths=0, zorder=1,transform=geodetic)
        
        """
        Overplotting the FoV of GOTO
        """
        col = ["id","runid","fov","target"]
        fov = pd.read_table("Fermi_fov.txt", header=None, skiprows=2, sep="|", names=col)
        fov.dropna(inplace=True)
        fov = fov[fov["target"].str.contains("Fermi_"+trigno)]
        for fv in fov["fov"].values:
            corners = fv.split("),(")
            c = []
            c.append(corners[0].split("(")[-1].split(","))
            c.append(corners[1].split(","))
            c.append(corners[2].split(","))
            c.append(corners[3].split(")")[0].split(","))
            for i in np.arange(4):
                c[i] = [float(k) for k in c[i]]
            tilelist = [list(i) for i in zip(*c)]
            ra, dec = smt.getshape(tilelist, steps=10)
            acolor = colorConverter.to_rgba('#C44677', alpha=0)
            axes.fill(ra, dec, fill=True, facecolor=acolor,
                      linewidth=1, linestyle='solid',
                      edgecolor='black', transform=geodetic)

In [14]:
plot_skymap('566755024', 330.56, 29.9, 4.123)